In [1]:
import pandas as pd
import numpy as np

In [12]:
train = pd.read_csv("train.csv",dtype = {'Upc': str, 'FinelineNumber':str, 'TripType':"category" , 'Weekday':"category"})
test = pd.read_csv("test.csv",dtype = {'Upc': str, 'FinelineNumber':str, 'TripType':"category" , 'Weekday':"category"})

In [13]:
all_data = pd.concat([train,test])
all_data.reset_index(inplace=True,drop=True)

In [14]:
all_data.drop(['TripType', 'Upc'],axis = 1,inplace = True)

In [15]:
all_data['FinelineNumber_copy'] = all_data['FinelineNumber']

In [16]:
group_visit_fineline = all_data.groupby(['VisitNumber','FinelineNumber'],as_index=False).agg({'FinelineNumber_copy':'count'})

In [ ]:
# day= pd.get_dummies(all_data["VisitNumber", "Weekday"], columns = ["Weekday"])
# # day.drop_duplicates()

In [18]:
dictVector = []
visitnum = group_visit_fineline['VisitNumber'][1]
visitdict = {}
for i in range(len(group_visit_fineline)):
    if(group_visit_fineline['VisitNumber'][i] == visitnum):
        visitdict[group_visit_fineline['FinelineNumber'][i]] = group_visit_fineline['FinelineNumber_copy'][i]
    else:
        dictVector.append(visitdict)
        visitdict = {}
        visitdict[group_visit_fineline['FinelineNumber'][i]] = group_visit_fineline['FinelineNumber_copy'][i]
        visitnum = group_visit_fineline['VisitNumber'][i]
# last one
dictVector.append(visitdict)

In [19]:
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer(sparse=False)

In [20]:
X = v.fit_transform(dictVector)

In [33]:
day = pd.get_dummies(all_data[["VisitNumber", "Weekday"]], columns = ["Weekday"])
day = day.drop_duplicates()

# find unique VisitNumber
uids = group_visit_fineline['VisitNumber'].unique()

# keep VisitNumber in uids
day = day[day['VisitNumber'].isin(uids)]

(188535, 8)

In [35]:
day.sort_values(['VisitNumber'], inplace=True)

In [36]:
new_X = np.concatenate( (X, day), axis=1)